In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

# To install fairseq from source and develop locally:
!git clone https://github.com/choyi0521/fairseq
%cd /content/fairseq
!pip install --editable .

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Cloning into 'fairseq'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 12169 (delta 0), reused 1 (delta 0), pack-reused 12164
Receiving objects: 100% (12169/12169), 5.69 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (8965/8965), done.
/content/fairseq
Obtaining file:///content/fairseq
     |████████████████████████████████| 645kB 2.8MB/s 
  Running setup.py deve

In [2]:
!git checkout dev5
!git pull origin dev5

Branch 'dev5' set up to track remote branch 'dev5' from 'origin'.
Switched to a new branch 'dev5'
From https://github.com/choyi0521/fairseq
 * branch            dev5       -> FETCH_HEAD
Already up to date.


In [3]:
# Download and prepare the data
%cd /content/fairseq
%cd examples/translation/
!bash prepare-iwslt14.sh
%cd ../..

# Preprocess/binarize the data
TEXT="examples/translation/iwslt14.tokenized.de-en"
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/iwslt14.tokenized.de-en \
    --workers 20

/content/fairseq
/content/fairseq/examples/translation
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 147464 (delta 104), reused 118 (delta 74), pack-reused 147295
Receiving objects: 100% (147464/147464), 129.73 MiB | 10.50 MiB/s, done.
Resolving deltas: 100% (113938/113938), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 538 (delta 10), reused 21 (delta 7), pack-reused 509
Receiving objects: 100% (538/538), 226.98 KiB | 626.00 KiB/s, done.
Resolving deltas: 100% (316/316), done.
--2019-11-28 16:07:54--  https://wit3.fbk.eu/archive/2014-01/texts/de/en/de-en.tgz
Resolving wit3.fbk.eu (wit3.fbk.eu)... 

In [3]:

SAVE="/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt"

# Training
%cd /content/fairseq
!mkdir -p "{SAVE}" 
!CUDA_VISIBLE_DEVICES=0 $(which fairseq-train) data-bin/iwslt14.tokenized.de-en \
    --encoder-conv-type ddynamic --decoder-conv-type ddynamic \
    --clip-norm 0 --optimizer adam --lr 0.0005 \
    --source-lang de --target-lang en --max-tokens 4000 --no-progress-bar \
    --log-interval 100 --min-lr '1e-09' --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --lr-scheduler inverse_sqrt \
    --ddp-backend=no_c10d \
    --max-update 50000 --warmup-updates 4000 --warmup-init-lr '1e-07' \
    --adam-betas '(0.9, 0.98)' --keep-last-epochs 10 \
    -a lightconv_iwslt_de_en --save-dir "{SAVE}" \
    --dropout 0.3 --attention-dropout 0.1 --weight-dropout 0.1 \
    --encoder-glu 0 --decoder-glu 0 \
    --encoder-gau 1 --decoder-gau 1 \
    --conv-mixed 0 \
    --encoder-attention-heads 4 --decoder-attention-heads 4 \
    --encoder-attention-proj-heads 1 --decoder-attention-proj-heads 1 \
    2>&1 | tee "{SAVE}/train0.log"

/content/fairseq
Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='lightconv_iwslt_de_en', attention_dropout=0.1, best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.0, conv_mixed=False, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data-bin/iwslt14.tokenized.de-en', dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=4, decoder_attention_proj_heads=1, decoder_conv_dim=512, decoder_conv_type='ddynamic', decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_gau=True, decoder_glu=False, decoder_input_dim=512, decoder_kernel_size_list=[3, 7, 15, 31, 31, 31], decoder_layers=6, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_wor

In [4]:
!python scripts/average_checkpoints.py --inputs "{SAVE}" \
    --num-epoch-checkpoints 10 --output "{SAVE}/checkpoint_last10_avg.pt"

# Evaluation
!CUDA_VISIBLE_DEVICES=0 fairseq-generate data-bin/iwslt14.tokenized.de-en --path "{SAVE}/checkpoint_last10_avg.pt" --batch-size 128 --beam 4 --remove-bpe --lenpen 1 --gen-subset test --quiet 

Namespace(checkpoint_upper_bound=None, inputs=['/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt'], num_epoch_checkpoints=10, num_update_checkpoints=None, output='/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint_last10_avg.pt')
averaging checkpoints:  ['/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint44.pt', '/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint43.pt', '/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint42.pt', '/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint41.pt', '/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint40.pt', '/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint39.pt', '/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint38.pt', '/content/gdrive/My Drive/colab/payless/save_gau/dynamic_conv_iwslt/checkpoint37.pt', '/content/gd